In [1]:
%load_ext autoreload
from include import *
from data import *
import builtins
from Cope import debug
data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/cereal.csv')
# For when we need a lot of samples
bigData = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

relevant = ('mfr', 'fat', 'calories', 'name', 'rating')
target = 'rating'

In [2]:
data

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3,60,25,3,1.0,0.75,39.106174
73,Trix,G,C,110,1,1,140,0.0,13.0,12,25,25,2,1.0,1.00,27.753301
74,Wheat Chex,R,C,100,3,1,230,3.0,17.0,3,115,25,1,1.0,0.67,49.787445
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3,110,25,1,1.0,1.00,51.592193


In [15]:
def split(*data, amt:Union[int, float, Iterable[Union[int, float]]]=.2, method:Union[Literal['random', 'head', 'tail'], Iterable[Literal['train', 'test']]]='random', target=[], splitTargets=False, seed=42):
    """ Splits the given data, both into train/test sets, and by taking out targets at the same time
        `target` can be a string or an iterable
        If `splitTargets` is set to False, the targets will always return DataFrames, even if
            they only have 1 column
        If you pass in multiple items for data, AND specify a target feature[s], then all the items
            must have the target columns
        The order goes:
            train_X, test_X, train_X1, test_X1, ..., train_y, test_y, train_y1, test_y1
            where it continues adding data and target splits in the order they are given.
            Simply put, it outputs in the same order you input the parameters as much as possible.
            Don't give multiple data AND split targets at the same time. While it can do it,
                it's simply too confusing to think through the order of the returned parameters.
        Setting the `method` to 'chunk' is the same as setting it to 'tail'.
    """
    if len(ensureIterable(data)) > 1 and len(target):
        warn("Please don't give multiple data AND split targets at the same time. While it can do it, "
             "it's simply too confusing to think through the order of the returned parameters.")
    # Pop the targets and combine everything into 1 ordered list of things we need to split
    splitMe = []
    for d in ensureIterable(data):
        d = d.copy()

        targets = [d.pop(t) for t in ensureIterable(target)]
        if splitTargets:
            splitMe += targets
        else:
            splitMe.append(pd.DataFrame(dict(zip(ensureIterable(target), targets))))
        # It makes more sense to do data, then target, not target then data
        splitMe.insert(0 if len(targets) else len(splitMe), d)

    # Now split everything in the list (order is important!)
    if method == 'random':
        disp(splitMe)
        return skms.train_test_split(*splitMe, test_size=amt, random_state=seed)
    elif method in ('head', 'tail', 'chunk'):
        rtn = []
        for d in splitMe:
            # Head an tail splitting are the same, just with opposite amts
            split = round(len(d) * (amt if method == 'head' else (1-amt)))
            rtn += [d.iloc[:split], d.iloc[split:]]
        return rtn
    else:
        raise TypeError(f"Invalid method parameter given")


In [16]:
# train_X, test_X, train_X1, test_X1, ..., train_y, test_y, train_y1, test_y1
_testSize = len(data) * .2
_trainSize = len(data) * .8
_test_data, _train_data = skms.train_test_split(data, test_size=.2, random_state=42)

try:
    split(data, amt=[.2])
except: print('success')
else: print('failed to throw error')
try:
    split(data, amt=(.6), method=('train', 'test', 'train'))
except: print('success')
else: print('failed to throw error')
try:
    split(data, amt=(.2, .6), method=('train', 'test'))
except: print('success')
else: print('failed to throw error')

# split(data, amt=(.2, .6), method=('train', 'test', 'train'))
# split(data, amt=[.2], method=('train', 'test'))
# split(data, amt=.2, method='tail')
# split(data, amt=.2, method='head')
split(data, amt=.2, method='random')

# # split(data, amt=(.2, .6), method=('train', 'test', 'train'), target='mfr')
# # split(data, amt=[.2], method=('train', 'test'), target='mfr')
# split(data, amt=.2, method='tail', target='mfr')
# split(data, amt=.2, method='head', target='mfr')
# split(data, amt=.2, method='random', target='mfr')

# # split(data, amt=(.2, .6), method=('train', 'test', 'train'), target=['mfr', 'fat'])
# # split(data, amt=[.2], method=('train', 'test'), target=['mfr', 'fat'])
# split(data, amt=.2, method='tail', target=['mfr', 'fat'])
# split(data, amt=.2, method='head', target=['mfr', 'fat'])
# split(data, amt=.2, method='random', target=['mfr', 'fat'])

# # split(data, amt=(.2, .6), method=('train', 'test', 'train'), target=['mfr', 'fat'], splitTargets=True)
# # split(data, amt=[.2], method=('train', 'test'), target=['mfr', 'fat'], splitTargets=True)
# split(data, amt=.2, method='tail', target=['mfr', 'fat'], splitTargets=True)
# split(data, amt=.2, method='head', target=['mfr', 'fat'], splitTargets=True)
# split(data, amt=.2, method='random', target=['mfr', 'fat'], splitTargets=True)

{


""


,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3,60,25,3,1.0,0.75,39.106174
73,Trix,G,C,110,1,1,140,0.0,13.0,12,25,25,2,1.0,1.00,27.753301
74,Wheat Chex,R,C,100,3,1,230,3.0,17.0,3,115,25,1,1.0,0.67,49.787445
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3,110,25,1,1.0,1.00,51.592193


}
success
success
success
{


""


,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Triples,G,C,110,2,1,250,0.0,21.0,3,60,25,3,1.0,0.75,39.106174
73,Trix,G,C,110,1,1,140,0.0,13.0,12,25,25,2,1.0,1.00,27.753301
74,Wheat Chex,R,C,100,3,1,230,3.0,17.0,3,115,25,1,1.0,0.67,49.787445
75,Wheaties,G,C,100,3,1,200,3.0,17.0,3,110,25,1,1.0,1.00,51.592193


}


ValueError: Found input variables with inconsistent numbers of samples: [0, 77]